In [48]:
import pandas as pd
import yaml
import vectorbt as vbt
import numpy as np

def get_time_interval(config):
    start_date = pd.to_datetime(config['Backtesting_dates']['start']).normalize()
    end_date = pd.to_datetime(config['Backtesting_dates']['end']).normalize()
    return start_date, end_date

with open('config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    
df = pd.read_csv(config['Data_flname'], parse_dates=['Time'], index_col='Time')
df.drop(columns=['Volume'], inplace=True)
df.dropna(inplace=True)

roll = df['IV'].rolling(window=252, min_periods=252)
df['IVR'] = (df['IV'] - roll.min()) / (roll.max() - roll.min()) * 100

def calc_percentile(x):
        current_iv = x.iloc[-1]
        return (x < current_iv).mean() * 100

df['IVP %'] = roll.apply(calc_percentile, raw=False)
df['IVP/IVR blend'] = df['IVR'] * 0.5 + df['IVP %'] * 0.5
df.dropna(inplace=True)
start_date, end_date = get_time_interval(config)
df = df.loc[start_date:end_date]
df

,Open,High,Low,Close,IV,IVR,IVP %,IVP/IVR blend
Time,,,,,,,,
2015-01-05,27.07,27.16,26.35,26.56,0.335830,92.920182,99.206349,96.063266
2015-01-06,26.64,26.86,26.16,26.57,0.337508,93.892206,99.206349,96.549278
2015-01-07,26.80,27.05,26.67,26.94,0.332192,90.813697,97.619048,94.216372
2015-01-08,27.31,28.04,27.18,27.97,0.315952,81.409072,96.825397,89.117235
2015-01-09,28.17,28.31,27.55,28.00,0.319641,83.545381,96.825397,90.185389
...,...,...,...,...,...,...,...,...
2025-12-17,275.01,276.16,271.64,271.84,0.234379,7.807620,22.222222,15.014921
2025-12-18,273.61,273.63,266.95,272.19,0.233248,7.455805,19.841270,13.648537
2025-12-19,272.15,274.60,269.90,273.67,0.227232,5.583295,9.126984,7.355140


In [40]:
index_arr = df.index.to_numpy()
close_arr = df['Close'].to_numpy()
blend_arr = df['IVP/IVR blend'].to_numpy()
entry_arr = np.full(len(index_arr), False)
exit_arr = np.full(len(index_arr), False)
price_arr = np.full(len(index_arr), np.nan)

trade_is_open = False
ent_str = int(config['IVP/IVR blend']['start'].split('-')[0])
ent_end = int(config['IVP/IVR blend']['start'].split('-')[1])
ext_str = int(config['IVP/IVR blend']['end'].split('-')[0])
ext_end = int(config['IVP/IVR blend']['end'].split('-')[1])

for i in range(0, len(index_arr)):
    momentum_ok = abs(close_arr[i] - close_arr[i-5]) / close_arr[i] > 0.01
    if not trade_is_open and (ent_str <= blend_arr[i] <= ent_end) and momentum_ok:
        trade_is_open = True
        entry_arr[i] = True
        price_arr[i] = close_arr[i]
    elif trade_is_open and (ext_str <= blend_arr[i] <= ext_end):
        trade_is_open = False
        exit_arr[i] = True
        price_arr[i] = close_arr[i]

pf = vbt.Portfolio.from_signals(
    entries = entry_arr,
    exits = exit_arr,
    price = price_arr,
    open = df["Open"],
    close = close_arr,
    size = config['Trade']['size'],
    size_type = config['Trade']['size_type'],
    init_cash = config['Initial_cash'],
    freq = '1d'
    )

In [41]:
pf.stats()

Start                         2010-12-31 00:00:00
End                           2025-12-23 00:00:00
Period                         3768 days 00:00:00
Start Value                               10000.0
End Value                                 31865.0
Total Return [%]                           218.65
Benchmark Return [%]                  2264.236111
Max Gross Exposure [%]                  94.336599
Total Fees Paid                               0.0
Max Drawdown [%]                        12.323469
Max Drawdown Duration           421 days 00:00:00
Total Trades                                   42
Total Closed Trades                            41
Total Open Trades                               1
Open Trade PnL                             5911.0
Win Rate [%]                            48.780488
Best Trade [%]                          51.321222
Worst Trade [%]                         -9.555662
Avg Winning Trade [%]                   16.463045
Avg Losing Trade [%]                    -4.084133


In [42]:
pf.trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,Open,100.0,2010-12-31,11.52,0.0,2011-02-22,12.09,0.0,57.0,0.049479,Long,Closed,0
1,1,Open,100.0,2011-02-25,12.43,0.0,2011-03-18,11.81,0.0,-62.0,-0.049879,Long,Closed,1
2,2,Open,100.0,2011-03-28,12.52,0.0,2011-08-17,13.59,0.0,107.0,0.085463,Long,Closed,2
3,3,Open,100.0,2011-10-28,14.46,0.0,2011-11-01,14.16,0.0,-30.0,-0.020747,Long,Closed,3
4,4,Open,100.0,2012-01-06,15.09,0.0,2012-03-21,21.52,0.0,643.0,0.426110,Long,Closed,4
5,5,Open,100.0,2012-05-02,20.93,0.0,2012-05-17,18.93,0.0,-200.0,-0.095557,Long,Closed,5
6,6,Open,100.0,2012-06-19,20.98,0.0,2012-10-01,23.55,0.0,257.0,0.122498,Long,Closed,6
7,7,Open,100.0,2012-11-01,21.30,0.0,2012-11-12,19.39,0.0,-191.0,-0.089671,Long,Closed,7
8,8,Open,100.0,2013-01-24,16.09,0.0,2013-03-12,15.30,0.0,-79.0,-0.049099,Long,Closed,8
9,9,Open,100.0,2013-03-26,16.47,0.0,2013-04-02,15.35,0.0,-112.0,-0.068002,Long,Closed,9


In [43]:
pf.plot().show()